## Load modules

In [1]:
import numpy as np
import imp
import ens_snapshot_tools as ens
import matplotlib.pyplot as plt

# for debugging
# imp.reload(ens)

# Luke's
import utils #utils file with functions
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import numpy as np
import xarray as xr
import xesmf as xe
from scipy import stats
from scipy import signal
import time
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
%matplotlib inline
import cartopy.util as cutil
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.basemap import Basemap, maskoceans
import matplotlib.cm as cm

import utils

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

## Load control run

In [4]:
# Luke's system-specific file paths here

local_var = 'TS'

DATAPATH_cesm  =    '/Users/lukeaparsons/Documents/Data/CESM/CESM_LME/'+str(local_var)+'/'

SAVEPATH  =    '/Users/lukeaparsons/python/SEOFs/output/'
FIGUREPATH  =  '/Users/lukeaparsons/python/SEOFs/figures/'

In [5]:
#load control run to determine lat, lon, nt, etc.
ds = xr.open_dataset(DATAPATH_cesm + 'b.e11.B1850C5CN.f19_g16.0850cntl.001.cam.h0.TS.085001-184912.nc') #load tas
LAT = ds['lat'][:]
LON = ds['lon'][:]
ds_TS = ds['TS'][:,:,:]
ds_TS = ds_TS.groupby('time.year').mean('time')
ds_TS = ds_TS[0:1000,:,:]
ds_TS_input = ds_TS.values #use actual values as input data for later numpy calcs
[nt,nlat,nlon] = np.shape(ds_TS);
print(nt,nlat,nlon)

CESM_cntl_anoms = ds_TS - ds_TS.mean(axis=0)
CESM_cntl_ltm = ds_TS.mean(axis=0)
CESM_cntl_anoms_weighted = utils.weightdata(ds,CESM_cntl_anoms) #use function to latitude weight data
print('CESM control shape is: ',CESM_cntl_anoms_weighted.shape)

1000 96 144
CESM control shape is:  (1000, 96, 144)


In [ ]:
[u,s,vt] = np.linalg.svd(CESM_cntl_anoms_weighted.reshape(nt,nlon*nlat),full_matrices=False)
[nt,nlat,nlon] = np.shape(ds_TS);

time = CESM_cntl_anoms_weighted.year
lat = ds['lat'][:]
lon = ds['lon'][:]

# save output for sharing
np.savez(SAVEPATH + 'CESM_ctrl_wtd_SVD',
        u = u,
        s = s,
        vt = vt,
        lat = lat,
        lon = lon,
        time = time,
        nt = nt,
        nlat = nlat,
        nlon = nlon)

## Load LME

In [6]:
# find all files that have necessary start/end strings etc for the all-forcing run
filenames = sorted(os.listdir(DATAPATH_cesm))
filenames_cesm = {}
h = 0
for filename in sorted(os.listdir(DATAPATH_cesm)):
    if filename.endswith(".cam.h0.TS.085001-184912.nc"): #note this picks data 850-1849
        if filename.startswith("b.e11.BLMTRC5CN.f19_g16.0"): #note this string is the string for the all forcing data set
            #print("filename starts with: b.e11.BLMTRC5CN.f19_g16.0")
            #print(filename)
            filenames_cesm[h] = filename
            h = h + 1
filenames_cesm
nens = h

In [8]:
dat = np.empty([nt,nlat*nlon,nens])

for f in filenames_cesm:
    start0 = time.time() #keep track of time through for loop
    ds = xr.open_dataset(DATAPATH_cesm + filenames_cesm[f]) #load cesm data defined above
    lat = ds['lat'][:]
    lon = ds['lon'][:]
    data_TS = ds['TS'][:,:,:] - ds['TS'][:,:,:].mean(axis=0)
    data_TS1850 = data_TS.groupby('time.year').mean('time')[0:nt,:,:] #note that data includes first month of 1850, so exclude 1850!
    data_TS_anom_weighted = utils.weightdata(ds,data_TS1850) #use function to latitude weight data
    data_TS_ex = (data_TS_anom_weighted.values).reshape(nt,nlon*nlat) #note that xarray values must be exported to np.reshape
    dat[:,:,f] = data_TS_ex
    print("Done with loading, all calcs in ",round(time.time() - start0)," seconds")

Done with loading, all calcs in  2  seconds
Done with loading, all calcs in  4  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  5  seconds
Done with loading, all calcs in  7  seconds
Done with loading, all calcs in  7  seconds


In [20]:
[u,s,vt] = np.linalg.svd(dat,full_matrices=False)

ured  = u[:,:999]
vtred = vt[:999,:]
time = data_TS1850.year
lat = ds['lat'][:]
lon = ds['lon'][:]
nens = 13

# save output for sharing
np.savez(SAVEPATH + 'CESM_LME_all13_wtd_SVD',
        u = ured,
        s = s,
        vt = vtred,
        lat = lat,
        lon = lon,
        time = time,
        nt = nt,
        nlat = nlat,
        nlon = nlon,
        nens = nens)